In [23]:
import requests
import os
import json
import time
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta

# Load environment variables from .env file
load_dotenv()

# Retrieve API key from environment variable
api_key = os.getenv("RAPIDAPI_KEY")

if not api_key:
    print("ERROR: RAPIDAPI_KEY not found in .env file.")
    # You might want to raise an exception here or handle it appropriately
else:
    print("API Key loaded successfully.")

# Setup request headers
headers = {
    "x-rapidapi-key": api_key,
    "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

# Define the base URL
base_url = "https://aerodatabox.p.rapidapi.com"

API Key loaded successfully.


In [2]:
# These are examples and may change seasonally or otherwise.
target_flights = [
    # --- Emirates ---
    {"airline": "Emirates", "flight_number": "EK600", "origin_hub": "DXB", "destination_icao": "OPKC", "destination_city": "Karachi"},
    {"airline": "Emirates", "flight_number": "EK622", "origin_hub": "DXB", "destination_icao": "OPLA", "destination_city": "Lahore"},
    {"airline": "Emirates", "flight_number": "EK612", "origin_hub": "DXB", "destination_icao": "OPIS", "destination_city": "Islamabad"},
    # --- Qatar Airways ---
    {"airline": "Qatar Airways", "flight_number": "QR610", "origin_hub": "DOH", "destination_icao": "OPKC", "destination_city": "Karachi"},
    {"airline": "Qatar Airways", "flight_number": "QR620", "origin_hub": "DOH", "destination_icao": "OPLA", "destination_city": "Lahore"},
    {"airline": "Qatar Airways", "flight_number": "QR632", "origin_hub": "DOH", "destination_icao": "OPIS", "destination_city": "Islamabad"},
    # --- Turkish Airlines ---
    {"airline": "Turkish Airlines", "flight_number": "TK708", "origin_hub": "IST", "destination_icao": "OPKC", "destination_city": "Karachi"},
    {"airline": "Turkish Airlines", "flight_number": "TK714", "origin_hub": "IST", "destination_icao": "OPLA", "destination_city": "Lahore"},
    {"airline": "Turkish Airlines", "flight_number": "TK710", "origin_hub": "IST", "destination_icao": "OPIS", "destination_city": "Islamabad"},
    # --- Etihad Airways ---
    {"airline": "Etihad Airways", "flight_number": "EY200", "origin_hub": "AUH", "destination_icao": "OPKC", "destination_city": "Karachi"}, # Example, verify number
    {"airline": "Etihad Airways", "flight_number": "EY247", "origin_hub": "AUH", "destination_icao": "OPLA", "destination_city": "Lahore"}, # Example, verify number
    {"airline": "Etihad Airways", "flight_number": "EY231", "origin_hub": "AUH", "destination_icao": "OPIS", "destination_city": "Islamabad"}, # Example, verify number
    # Add other airlines like Saudia, PIA (if applicable and data available)
]

print(f"Defined {len(target_flights)} target flight legs.")

Defined 12 target flight legs.


In [3]:
single_flight=target_flights[7]
single_flight

{'airline': 'Turkish Airlines',
 'flight_number': 'TK714',
 'origin_hub': 'IST',
 'destination_icao': 'OPLA',
 'destination_city': 'Lahore'}

In [4]:
def fetch_delay_stats(flight_number):
    """Fetches delay statistics for a given flight number."""
    url = f"{base_url}/flights/{flight_number}/delays"
    try:
        response = requests.get(url, headers=headers, timeout=15) # Add timeout
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        print(f"Successfully fetched data for {flight_number}. Status: {response.status_code}")
        return response.json()
    except requests.exceptions.Timeout:
        print(f"ERROR: Timeout fetching data for {flight_number}")
        return None
    except requests.exceptions.HTTPError as http_err:
        print(f"ERROR: HTTP error fetching data for {flight_number}: {http_err} - Response: {response.text}")
        return None
    except requests.exceptions.RequestException as req_err:
        print(f"ERROR: Request exception fetching data for {flight_number}: {req_err}")
        return None
    except json.JSONDecodeError:
        print(f"ERROR: Failed to decode JSON response for {flight_number}")
        return None

In [5]:
 
api_data = fetch_delay_stats('QR610')

Successfully fetched data for TK714. Status: 200


In [6]:
api_data

{'number': 'TK 714',
 'origins': [{'airportIcao': 'LTFM',
   'class': 'FlightAndHour',
   'scheduledHourUtc': 14,
   'medianDelay': '00:00:00',
   'delayPercentiles': [{'percentile': 5, 'delay': '00:00:00'},
    {'percentile': 10, 'delay': '00:00:00'},
    {'percentile': 15, 'delay': '00:00:00'},
    {'percentile': 20, 'delay': '00:00:00'},
    {'percentile': 25, 'delay': '00:00:00'},
    {'percentile': 30, 'delay': '00:00:00'},
    {'percentile': 35, 'delay': '00:00:00'},
    {'percentile': 40, 'delay': '00:00:00'},
    {'percentile': 45, 'delay': '00:00:00'},
    {'percentile': 50, 'delay': '00:00:00'},
    {'percentile': 55, 'delay': '00:00:00'},
    {'percentile': 60, 'delay': '00:01:00'},
    {'percentile': 65, 'delay': '00:10:00'},
    {'percentile': 70, 'delay': '00:10:00'},
    {'percentile': 75, 'delay': '00:21:00'},
    {'percentile': 80, 'delay': '00:36:00'},
    {'percentile': 85, 'delay': '00:56:00'},
    {'percentile': 90, 'delay': '01:21:00'},
    {'percentile': 95, 'del

In [17]:
def analyze_flight_stats(flight_data):
    """
    Extracts important delay statistics for a flight from the provided data.
    
    Args:
        flight_data (dict): Flight data in the format provided by the API.
        
    Returns:
        dict: Summary of flight delay statistics.
    """
    # Extract flight number
    flight_number = flight_data['number']
    
    # Initialize results dictionary
    results = {
        'flight_number': flight_number,
        'departure_options': [],
        'overall': {}
    }
    
    total_flights = 0
    all_from_dates = []
    all_to_dates = []
    weighted_delay_sum = 0
    
    # Process each origin (different departure hours)
    for origin in flight_data['origins']:
        airport = origin['airportIcao']
        hour = origin['scheduledHourUtc']
        flights_analyzed = origin['numConsideredFlights']
        total_flights += flights_analyzed
        
        # Extract date range
        from_date = origin['fromUtc']
        to_date = origin['toUtc']
        all_from_dates.append(from_date)
        all_to_dates.append(to_date)
        
        # Calculate delay percentages
        on_time_percentage = 0
        slight_delay = 0   # 15-30 minutes
        moderate_delay = 0  # 30-60 minutes
        severe_delay = 0   # > 60 minutes
        
        for bracket in origin['numFlightsDelayedBrackets']:
            if 'delayedFrom' in bracket and 'delayedTo' in bracket:
                # On-time flights (-15 min to +15 min)
                if bracket['delayedFrom'] == '-00:15:00' and bracket['delayedTo'] == '00:15:00':
                    on_time_percentage = bracket['percentage'] * 100
                # Slight delay (15-30 min)
                elif bracket['delayedFrom'] == '00:15:00' and bracket['delayedTo'] == '00:30:00':
                    slight_delay = bracket['percentage'] * 100
                # Moderate delay (30-60 min)
                elif bracket['delayedFrom'] == '00:30:00' and bracket['delayedTo'] == '01:00:00':
                    moderate_delay = bracket['percentage'] * 100
                # Severe delay (60+ min)
                elif bracket['delayedFrom'] == '01:00:00' and bracket['delayedTo'] == '02:00:00':
                    severe_delay += bracket['percentage'] * 100
            # Extreme delay (2+ hours)
            elif 'delayedFrom' in bracket and bracket['delayedFrom'] == '02:00:00':
                severe_delay += bracket['percentage'] * 100
        
        # Calculate total delayed percentage (flights delayed > 15 min)
        delayed_percentage = 100 - on_time_percentage
        weighted_delay_sum += delayed_percentage * flights_analyzed
        
        # Get the median delay and 90th percentile
        median_delay = origin['medianDelay']
        percentile_90 = next((p['delay'] for p in origin['delayPercentiles'] if p['percentile'] == 90), 'Unknown')
        
        # Add to results
        results['departure_options'].append({
            'airport': airport,
            'hour_utc': hour,
            'flights_analyzed': flights_analyzed,
            'date_range': f"{from_date} to {to_date}",
            'delayed_percentage': round(delayed_percentage, 1),
            'on_time_percentage': round(on_time_percentage, 1),
            'delay_buckets': {
                'slight_delay_15_30min': round(slight_delay, 1),
                'moderate_delay_30_60min': round(moderate_delay, 1),
                'severe_delay_60min_plus': round(severe_delay, 1)
            },
            'median_delay': median_delay,
            '90th_percentile_delay': percentile_90
        })
    
    # Calculate overall statistics
    earliest_date = min(all_from_dates) if all_from_dates else "Unknown"
    latest_date = max(all_to_dates) if all_to_dates else "Unknown"
    overall_delayed_percentage = (weighted_delay_sum / total_flights) if total_flights > 0 else 0
    
    results['overall'] = {
        'total_flights_analyzed': total_flights,
        'overall_date_range': f"{earliest_date} to {latest_date}",
        'overall_delayed_percentage': round(overall_delayed_percentage, 1)
    }
    
    return results

In [18]:

parsed_data = analyze_flight_stats(api_data)


In [19]:
parsed_data

{'flight_number': 'TK 714',
 'departure_options': [{'airport': 'LTFM',
   'hour_utc': 14,
   'flights_analyzed': 38,
   'date_range': '2024-05-17 14:35 to 2024-08-15 14:35',
   'delayed_percentage': 26.3,
   'on_time_percentage': 73.7,
   'delay_buckets': {'slight_delay_15_30min': 5.3,
    'moderate_delay_30_60min': 5.3,
    'severe_delay_60min_plus': 15.8},
   'median_delay': '00:00:00',
   '90th_percentile_delay': '01:21:00'},
  {'airport': 'LTFM',
   'hour_utc': 17,
   'flights_analyzed': 90,
   'date_range': '2024-04-09 17:55 to 2024-07-08 17:55',
   'delayed_percentage': 21.1,
   'on_time_percentage': 78.9,
   'delay_buckets': {'slight_delay_15_30min': 4.4,
    'moderate_delay_30_60min': 12.2,
    'severe_delay_60min_plus': 4.4},
   'median_delay': '00:00:00',
   '90th_percentile_delay': '00:40:00'},
  {'airport': 'LTFM',
   'hour_utc': 18,
   'flights_analyzed': 59,
   'date_range': '2023-12-31 18:15 to 2024-03-30 18:15',
   'delayed_percentage': 11.9,
   'on_time_percentage': 88

In [20]:
   # Print summary
print(f"Flight: {parsed_data['flight_number']}")
print(f"Total flights analyzed: {parsed_data['overall']['total_flights_analyzed']}")
print(f"Date range: {parsed_data['overall']['overall_date_range']}")
print(f"Overall delay percentage: {parsed_data['overall']['overall_delayed_percentage']}%")

print("\nDelay details by departure time:")
for option in parsed_data['departure_options']:
    print(f"\n  Departure hour (UTC): {option['hour_utc']}:00")
    print(f"  Flights analyzed: {option['flights_analyzed']}")
    print(f"  Delayed flights: {option['delayed_percentage']}%")
    print(f"  Delay breakdown:")
    print(f"    - Slight (15-30 min): {option['delay_buckets']['slight_delay_15_30min']}%")
    print(f"    - Moderate (30-60 min): {option['delay_buckets']['moderate_delay_30_60min']}%")
    print(f"    - Severe (60+ min): {option['delay_buckets']['severe_delay_60min_plus']}%")

Flight: TK 714
Total flights analyzed: 187
Date range: 2023-12-31 18:15 to 2024-08-15 14:35
Overall delay percentage: 19.3%

Delay details by departure time:

  Departure hour (UTC): 14:00
  Flights analyzed: 38
  Delayed flights: 26.3%
  Delay breakdown:
    - Slight (15-30 min): 5.3%
    - Moderate (30-60 min): 5.3%
    - Severe (60+ min): 15.8%

  Departure hour (UTC): 17:00
  Flights analyzed: 90
  Delayed flights: 21.1%
  Delay breakdown:
    - Slight (15-30 min): 4.4%
    - Moderate (30-60 min): 12.2%
    - Severe (60+ min): 4.4%

  Departure hour (UTC): 18:00
  Flights analyzed: 59
  Delayed flights: 11.9%
  Delay breakdown:
    - Slight (15-30 min): 5.1%
    - Moderate (30-60 min): 5.1%
    - Severe (60+ min): 1.7%


In [29]:
def fetch_flight_delay(flight_number,days_back=7):
    """Fetches delay statistics for a given flight number."""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back)
    
    start_str = start_date.strftime("%Y-%m-%d")
    end_str = end_date.strftime("%Y-%m-%d")
    
    print(f"Fetching flight data for {flight_number} from {start_str} to {end_str}")
    url = f"{base_url}/flights/number/{flight_number}/{start_str}/{end_str}?dateLocalRole=Both"
    try:
        response = requests.get(url, headers=headers, timeout=15) # Add timeout
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        print(f"Successfully fetched data for {flight_number}. Status: {response.status_code}")
        return response.json()
    except requests.exceptions.Timeout:
        print(f"ERROR: Timeout fetching data for {flight_number}")
        return None
    except requests.exceptions.HTTPError as http_err:
        print(f"ERROR: HTTP error fetching data for {flight_number}: {http_err} - Response: {response.text}")
        return None
    except requests.exceptions.RequestException as req_err:
        print(f"ERROR: Request exception fetching data for {flight_number}: {req_err}")
        return None
    except json.JSONDecodeError:
        print(f"ERROR: Failed to decode JSON response for {flight_number}")
        return None

In [30]:
flight_recent_response = fetch_flight_delay(single_flight['flight_number'])

Fetching flight data for TK714 from 2025-03-21 to 2025-03-28
Successfully fetched data for TK714. Status: 200


In [31]:
flight_recent_response

[{'greatCircleDistance': {'meter': 4184796.33,
   'km': 4184.8,
   'mile': 2600.31,
   'nm': 2259.61,
   'feet': 13729646.74},
  'departure': {'airport': {'icao': 'LTFM',
    'iata': 'IST',
    'name': 'Istanbul',
    'shortName': 'Istanbul',
    'municipalityName': 'Istanbul',
    'location': {'lat': 41.275276, 'lon': 28.751944},
    'countryCode': 'TR',
    'timeZone': 'Europe/Istanbul'},
   'scheduledTime': {'utc': '2025-03-20 18:20Z',
    'local': '2025-03-20 21:20+03:00'},
   'revisedTime': {'utc': '2025-03-20 18:30Z',
    'local': '2025-03-20 21:30+03:00'},
   'checkInDesk': 'E-F',
   'gate': 'B14',
   'quality': ['Basic', 'Live']},
  'arrival': {'airport': {'icao': 'OPLA',
    'iata': 'LHE',
    'name': 'Lahore Alama Iqbal',
    'shortName': 'Alama Iqbal',
    'municipalityName': 'Lahore',
    'location': {'lat': 31.5216, 'lon': 74.4036},
    'countryCode': 'PK',
    'timeZone': 'Asia/Karachi'},
   'scheduledTime': {'utc': '2025-03-20 23:35Z',
    'local': '2025-03-21 04:35+05:0

In [35]:
def calculate_delay_stats(flight_data,include_predictions=True):
    """
    Calculates delay statistics from flight tracking data.
    Works with data from any airline or flight number.
    
    Args:
        flight_data (list): List of flight tracking data objects from AeroDataBox API.
        
    Returns:
        dict: Summary of flight delay statistics.
    """
    if not flight_data:
        return {"error": "No flight data provided"}
    
    # Extract flight number from first record
    flight_number = flight_data[0].get("number", "Unknown")
    airline_name = flight_data[0].get("airline", {}).get("name", "Unknown")
    
    # Extract route information
    departure_airport = flight_data[0].get("departure", {}).get("airport", {})
    arrival_airport = flight_data[0].get("arrival", {}).get("airport", {})
    route = f"{departure_airport.get('iata', '')} → {arrival_airport.get('iata', '')}"
    
    # Initialize result dictionary
    result = {
        "flight_number": flight_number,
        "airline": airline_name,
        "route": route,
        "total_flights": len(flight_data),
        "date_range": "",
        "individual_flights": [],
        "delay_statistics": {
            "departure": {
                "average_delay_minutes": 0,
                "median_delay_minutes": 0,
                "on_time_percentage": 0,
                "delayed_percentage": 0,
                "delay_buckets": {
                    "slight_delay_15_30min": 0,
                    "moderate_delay_30_60min": 0,
                    "severe_delay_60min_plus": 0
                }
            },
            "arrival": {
                "average_delay_minutes": 0,
                "median_delay_minutes": 0,
                "on_time_percentage": 0,
                "delayed_percentage": 0,
                "delay_buckets": {
                    "slight_delay_15_30min": 0,
                    "moderate_delay_30_60min": 0,
                    "severe_delay_60min_plus": 0
                }
            }
        }
    }
    
    # Calculate date range
    dates = []
    departure_delays = []
    arrival_delays = []
    
    # Process each flight
    for flight in flight_data:
        try:
            # Get departure date for date range
            departure_scheduled = safe_get(flight, ["departure", "scheduledTime", "utc"])
            scheduled_datetime = None
            if departure_scheduled:
                scheduled_datetime = parse_time(departure_scheduled)
                if scheduled_datetime:
                    dates.append(scheduled_datetime.date())
            
            # Calculate departure delay
            departure_info = flight.get("departure", {})
            scheduled_departure = safe_get(flight, ["departure", "scheduledTime", "utc"])
            
            # Try different time fields for actual departure
            actual_departure = None
            if safe_get(flight, ["departure", "runwayTime", "utc"]):
                actual_departure = safe_get(flight, ["departure", "runwayTime", "utc"])
            elif safe_get(flight, ["departure", "revisedTime", "utc"]):
                actual_departure = safe_get(flight, ["departure", "revisedTime", "utc"])
            
            departure_delay_minutes = 0
            if scheduled_departure and actual_departure:
                scheduled_dt = parse_time(scheduled_departure)
                actual_dt = parse_time(actual_departure)
                if scheduled_dt and actual_dt:
                    departure_delay = actual_dt - scheduled_dt
                    departure_delay_minutes = departure_delay.total_seconds() / 60
                    departure_delays.append(departure_delay_minutes)
            
            # Calculate arrival delay
            arrival_info = flight.get("arrival", {})
            scheduled_arrival = safe_get(flight, ["arrival", "scheduledTime", "utc"])
            
            # Try different time fields for actual arrival
            actual_arrival = None
            is_predicted = False
            if safe_get(flight, ["arrival", "runwayTime", "utc"]):
                actual_arrival = safe_get(flight, ["arrival", "runwayTime", "utc"])
            elif safe_get(flight, ["arrival", "revisedTime", "utc"]):
                actual_arrival = safe_get(flight, ["arrival", "revisedTime", "utc"])
            elif safe_get(flight, ["arrival", "predictedTime", "utc"]):
                actual_arrival = safe_get(flight, ["arrival", "predictedTime", "utc"])
                is_predicted = True
            
            arrival_delay_minutes = 0
            if scheduled_arrival and actual_arrival:
                scheduled_dt = parse_time(scheduled_arrival)
                actual_dt = parse_time(actual_arrival)
                if scheduled_dt and actual_dt:
                    arrival_delay = actual_dt - scheduled_dt
                    arrival_delay_minutes = arrival_delay.total_seconds() / 60
                    # Only count non-predicted times in statistics
                    if not is_predicted or include_predictions:
                        arrival_delays.append(arrival_delay_minutes)
                    #if not is_predicted:
                    #    arrival_delays.append(arrival_delay_minutes)
            
            # Flight status 
            status = flight.get("status", "Unknown")
            
            # Get aircraft details safely
            aircraft_model = safe_get(flight, ["aircraft", "model"], "")
            aircraft_reg = safe_get(flight, ["aircraft", "reg"], "")
            
            aircraft_info = aircraft_model
            if aircraft_reg:
                aircraft_info += f" ({aircraft_reg})"
            
            # Get local time for display
            local_scheduled_dep = safe_get(flight, ["departure", "scheduledTime", "local"], "")
            local_actual_dep = (
                safe_get(flight, ["departure", "runwayTime", "local"]) or 
                safe_get(flight, ["departure", "revisedTime", "local"], "")
            )
            
            local_scheduled_arr = safe_get(flight, ["arrival", "scheduledTime", "local"], "")
            local_actual_arr = (
                safe_get(flight, ["arrival", "runwayTime", "local"]) or 
                safe_get(flight, ["arrival", "revisedTime", "local"]) or
                safe_get(flight, ["arrival", "predictedTime", "local"], "")
            )
            
            # Add note for predicted times
            arrival_note = " (predicted)" if is_predicted else ""
            
            # Format airport names
            dep_airport_str = f"{safe_get(departure_info, ['airport', 'iata'], '')} ({safe_get(departure_info, ['airport', 'name'], '')})"
            arr_airport_str = f"{safe_get(arrival_info, ['airport', 'iata'], '')} ({safe_get(arrival_info, ['airport', 'name'], '')})"
            
            # Add to individual flights list
            result["individual_flights"].append({
                "date": scheduled_datetime.strftime("%Y-%m-%d") if scheduled_datetime else "Unknown",
                "status": status,
                "departure": {
                    "airport": dep_airport_str,
                    "scheduled": local_scheduled_dep,
                    "actual": local_actual_dep,
                    "delay_minutes": round(departure_delay_minutes, 1),
                    "terminal": departure_info.get("terminal", ""),
                    "gate": departure_info.get("gate", "")
                },
                "arrival": {
                    "airport": arr_airport_str,
                    "scheduled": local_scheduled_arr,
                    "actual": f"{local_actual_arr}{arrival_note}",
                    "delay_minutes": round(arrival_delay_minutes, 1),
                    "terminal": arrival_info.get("terminal", "")
                },
                "aircraft": aircraft_info
            })
            
        except Exception as e:
            # Skip flights with parsing errors
            print(f"Error processing flight: {e}")
            continue
    
    # Calculate date range
    if dates:
        min_date = min(dates).strftime("%Y-%m-%d")
        max_date = max(dates).strftime("%Y-%m-%d")
        result["date_range"] = f"{min_date} to {max_date}"
    
    # Calculate departure delay statistics
    if departure_delays:
        result["delay_statistics"]["departure"] = calculate_statistics(departure_delays)
    
    # Calculate arrival delay statistics
    if arrival_delays:
        result["delay_statistics"]["arrival"] = calculate_statistics(arrival_delays)
    
    return result

def safe_get(data, keys, default=None):
    """
    Safely get a nested value from a dictionary.
    
    Args:
        data (dict): Dictionary to extract value from
        keys (list): List of keys to navigate the nested structure
        default: Value to return if the path doesn't exist
        
    Returns:
        The value at the specified path or the default value
    """
    current = data
    for key in keys:
        if isinstance(current, dict) and key in current:
            current = current[key]
        else:
            return default
    return current

def parse_time(time_str):
    """
    Parse a time string to a datetime object, handling different formats.
    
    Args:
        time_str (str): Time string to parse
        
    Returns:
        datetime: Parsed datetime object or None if parsing fails
    """
    if not time_str:
        return None
    
    # Remove Z suffix if present
    if time_str.endswith('Z'):
        time_str = time_str[:-1]
    
    # Try different datetime formats
    formats = [
        "%Y-%m-%d %H:%M",  # 2025-01-01 07:55
        "%Y-%m-%d %H:%M:%S"  # 2025-01-01 07:55:00
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(time_str, fmt)
        except ValueError:
            continue
    
    return None

def calculate_statistics(delays):
    """
    Calculate statistics for a list of delays.
    
    Args:
        delays (list): List of delay values in minutes
        
    Returns:
        dict: Delay statistics
    """
    if not delays:
        return {
            "average_delay_minutes": 0,
            "median_delay_minutes": 0,
            "on_time_percentage": 0,
            "delayed_percentage": 0,
            "delay_buckets": {
                "slight_delay_15_30min": 0,
                "moderate_delay_30_60min": 0,
                "severe_delay_60min_plus": 0
            }
        }
    
    # Sort delays for percentile calculations
    sorted_delays = sorted(delays)
    
    # Average delay
    avg_delay = sum(delays) / len(delays)
    
    # Median delay
    middle = len(sorted_delays) // 2
    if len(sorted_delays) % 2 == 0:
        med_delay = (sorted_delays[middle-1] + sorted_delays[middle]) / 2
    else:
        med_delay = sorted_delays[middle]
    
    # On-time percentage (less than 15 minutes delay)
    on_time_count = sum(1 for delay in delays if delay < 15)
    on_time_pct = on_time_count / len(delays) * 100
    delayed_pct = 100 - on_time_pct
    
    # Delay buckets
    slight_delay = sum(1 for delay in delays if 15 <= delay < 30)
    moderate_delay = sum(1 for delay in delays if 30 <= delay < 60)
    severe_delay = sum(1 for delay in delays if delay >= 60)
    
    slight_delay_pct = slight_delay / len(delays) * 100
    moderate_delay_pct = moderate_delay / len(delays) * 100
    severe_delay_pct = severe_delay / len(delays) * 100
    
    return {
        "average_delay_minutes": round(avg_delay, 1),
        "median_delay_minutes": round(med_delay, 1),
        "on_time_percentage": round(on_time_pct, 1),
        "delayed_percentage": round(delayed_pct, 1),
        "delay_buckets": {
            "slight_delay_15_30min": round(slight_delay_pct, 1),
            "moderate_delay_30_60min": round(moderate_delay_pct, 1),
            "severe_delay_60min_plus": round(severe_delay_pct, 1)
        }
    }

In [36]:
stats = calculate_delay_stats(flight_recent_response)

In [37]:
stats

{'flight_number': 'TK 714',
 'airline': 'Turkish Airlines',
 'route': 'IST → LHE',
 'total_flights': 9,
 'date_range': '2025-03-20 to 2025-03-28',
 'individual_flights': [{'date': '2025-03-20',
   'status': 'Departed',
   'departure': {'airport': 'IST (Istanbul)',
    'scheduled': '2025-03-20 21:20+03:00',
    'actual': '2025-03-20 21:30+03:00',
    'delay_minutes': 10.0,
    'terminal': '',
    'gate': 'B14'},
   'arrival': {'airport': 'LHE (Lahore Alama Iqbal)',
    'scheduled': '2025-03-21 04:35+05:00',
    'actual': '2025-03-21 05:32+05:00 (predicted)',
    'delay_minutes': 57.0,
    'terminal': 'M'},
   'aircraft': 'Airbus A330-300'},
  {'date': '2025-03-21',
   'status': 'Departed',
   'departure': {'airport': 'IST (Istanbul)',
    'scheduled': '2025-03-21 21:20+03:00',
    'actual': '2025-03-21 21:20+03:00',
    'delay_minutes': 0.0,
    'terminal': '',
    'gate': 'D16'},
   'arrival': {'airport': 'LHE (Lahore Alama Iqbal)',
    'scheduled': '',
    'actual': '2025-03-22 05:22+